# MATH50003 Numerical Analysis (2021–2022) Practice Computer-based Exam


For each problem, replace the `# TODO` to complete the question.
The unit tests are provided to help you test your answers.
You have 1 hour to complete the exam, as well as 1 hour for downloading/uploading.

Problems are marked A/B/C to indicate difficulty ("A" being most difficult).
Partial credit will be awarded for reasonable attempts even if the tests
are not passed.

You may use existing code from anywhere
but you are **REQUIRED** to cite the source if it is not part of the module material,
ideally by including a weblink in a comment. You **MUST NOT** ask for help online or
communicate with others within or outside the module.
Failure to follow these rules will be considered misconduct.



You should use the following packages:

In [1]:
using LinearAlgebra, SetRounding, Test

**WARNING** It may be necessary to restart the kernel if issues arise. Remember to reload the packages
when you do so.

## 1. Numbers

**Problem 1.1 (C)** Complete the following function `divideby3(x)` that
returns a tuple `a,b` such that `a` is the largest `Float64` less 
than or equal to `x/3` and `b` is the smallest `Float64` greater than or equal to `x/3`.

In [14]:
function divideby3(x)
    # TODO: assign a,b so that a ≤ x ≤ b where b is either equal to a or the next float
    x = Float64(x)
    a = setrounding(Float64, RoundDown) do
        x/3
    end
    b = setrounding(Float64, RoundUp) do
        b = x/3
    end
    a,b
end

divideby3 (generic function with 1 method)

In [15]:
x = 0.1 # arbitary x
a,b = divideby3(x)
@test a ≤ big(x)/3 ≤ b
@test b == a || b == nextfloat(a)

Test Passed

## 2. Differentiation

**Problem 2.1 (C)** Use the following off-center finite-difference approximation
$$
f'(x) ≈ {f(x+2h) - f(x-h) \over 3h}
$$
with an appropriately chosen $h$ to approximate
$$
f(x) = \cos(x^2)
$$
at $x = 0.1$ to 5 digits accuracy.

In [18]:
function fd(x)
    # TODO: implement a finite-difference rule 
    # to approximate f'(x)
    # for f(x) = cos(x^2)
    # with step-size h chosen to get sufficient accuracy

    h = sqrt(eps(Float64))

    f = x -> cos(x^2)

    diff = (f(x + 2*h)-f(x-h))/(3*h)
    
end

fd (generic function with 1 method)

In [19]:
@test abs(fd(0.1) + 2*0.1*sin(0.1^2)) ≤ 1E-5

Test Passed

**Problem 2.2 (A)** Consider a 2D version of a dual number:
$$
a + b ϵ_x + c ϵ_y
$$
such that
$$
ϵ_x^2 = ϵ_y^2 = ϵ_x ϵ_y =  0.
$$
Complete the following implementation supporting `+` and `*` (and
assuming `a,b,c` are `Float64`). Hint: you may need to work out on paper
how to multiply `(s.a + s.b ϵ_x + s.c ϵ_y)*(t.a + t.b ϵ_x + t.c ϵ_y)` using the
relationship above.

In [20]:
import Base: *, +, ^
struct Dual2D
    a::Float64
    b::Float64
    c::Float64
end


function +(s::Dual2D, t::Dual2D)
    Dual2D(s.a+t.a, s.b + t.b, s.c + t.c)
    ## TODO: Implement +, returning a Dual2D
end

function *(c::Number, s::Dual2D)
    ## TODO: Implement c * Dual2D(...), returning a Dual2D
    Dual2D(c*s.a, c*s.b, c*s.c)
    
end

function *(s::Dual2D, t::Dual2D)
    ## TODO: Implement Dual2D(...) * Dual2D(...), returning a Dual2D
    Dual2D(s.a*t.a, s.a*t.b + t.a*s.b, s.a*t.c + t.a*s.c)
    
    
end

* (generic function with 320 methods)

In [21]:
f = function (x, y) # (x+2y^2)^3 using only * and +
    z = (x + 2y * y)
    z * z * z
end

x,y = 1., 2.
@test f(Dual2D(x,1.,0.), Dual2D(y,0.,1.)) == Dual2D(f(x,y), 3(x+2y^2)^2, 12y*(x+2y^2)^2)

# This has computed the gradient as f(x,y) + f_x*ϵ_x + f_y*ϵ_y
# == (x+2y^2)^3 + 3(x+2y^2)^2*ϵ_x + 12y(x+2y^2)^2*ϵ_y

Test Passed

## 3. Structured Matrices

**Problem 3.1 (C)** Add an implementation of `inv(::PermutationMatrix)`
to return the inverse permutation as a `PermutationMatrix`. Hint: use
`invperm`.

In [22]:
import Base: getindex, size, *, inv

struct PermutationMatrix <: AbstractMatrix{Int}
    p::Vector{Int} # represents the permutation whose action is v[p]
    function PermutationMatrix(p::Vector)
        sort(p) == 1:length(p) || error("input is not a valid permutation")
        new(p)
    end
end

size(P::PermutationMatrix) = (length(P.p),length(P.p))
getindex(P::PermutationMatrix, k::Int, j::Int) = P.p[k] == j ? 1 : 0
*(P::PermutationMatrix, x::AbstractVector) = x[P.p]

function inv(P::PermutationMatrix)
    # TODO: return a PermutationMatrix representing the inverse permutation
    PermutationMatrix(invperm(P.p))
    
end

inv (generic function with 34 methods)

In [23]:
P = PermutationMatrix([3,4,2,1])
@test inv(P) isa PermutationMatrix
@test P*inv(P) == I

Test Passed

## 4. Decompositions

**Problem 4.1 (C)** For $𝐱 ∈ ℝ^n$ implement the reflection defined by
$$
\begin{align*}
𝐲 &:= 𝐱 + \|𝐱\| 𝐞_n \\
𝐰 &:= 𝐲/\|𝐲\| \\
Q_𝐱 &:= I - 2𝐰𝐰^⊤
\end{align*}
$$
in `lowerhouseholderreflection(x)`, which should return a `Matrix{Float64}`.
You may assume that `x` is a `Vector{Float64}`.

In [24]:
function lowerhouseholderreflection(x)
    ## TODO: implement the householder reflector defined above
    n = length(x)
    e_n = [zeros(n-1); 1]
    y = x + norm(x) * e_n
    w = y / norm(y)
    I - 2*w*w'
end

lowerhouseholderreflection (generic function with 1 method)

In [4]:
x = [1.0,2,3,4]
Q = lowerhouseholderreflection(x)
@test Q*x ≈ [zeros(3); -norm(x)]
@test Q'Q ≈ I
@test Q ≈ Q'

Test Passed
  Expression: Q ≈ Q'
   Evaluated: [0.9807354816671587 -0.03852903666568264 -0.05779355499852396 -0.18257418583505536; -0.03852903666568264 0.9229419266686347 -0.11558710999704792 -0.3651483716701107; -0.05779355499852396 -0.11558710999704792 0.8266193350044281 -0.5477225575051661; -0.18257418583505536 -0.3651483716701107 -0.5477225575051661 -0.7302967433402217] ≈ [0.9807354816671587 -0.03852903666568264 -0.05779355499852396 -0.18257418583505536; -0.03852903666568264 0.9229419266686347 -0.11558710999704792 -0.3651483716701107; -0.05779355499852396 -0.11558710999704792 0.8266193350044281 -0.5477225575051661; -0.18257418583505536 -0.3651483716701107 -0.5477225575051661 -0.7302967433402217]

**Problem 4.2 (A)** Complete the function `ql(A)` that
returns a QL decomposition, that is, `Q` is an orthogonal
matrix and `L` is lower triangular. You may assume that `A`
is a square `Matrix{Float64}`. Hint: use Problem 4.1 to lower triangularise.

In [25]:
function ql(A)
    m,n = size(A)
    m == n || error("not square")
    ## TODO Create Q and L such that Q'Q == I and L is lower triangular
    L = copy(A)
    Q = Matrix(1.0I, n, n)
    for j = n:-1:2 # last to 2nd column
        x = L[1:j,j]
        Qⱼ = lowerhouseholderreflection(x) # moves all of Qⱼ*x to the last entry
        L[1:j,1:j] = Qⱼ*L[1:j,1:j] # moves last column "down"
        # Q_1 * Q_2 * … * Q_j (note Q_k are symmetric, so this is inverse of Q_j * … * Q_1)
        # only changes the first j columns
        Q[:,1:j] = Q[:,1:j]*Qⱼ
    end
    Q, L
end

ql (generic function with 1 method)

In [26]:
ql(A)

UndefVarError: UndefVarError: A not defined

In [27]:
A = [1.0 2 3; 1 4 9; 1 1 1]
Q,L = ql(A)
@test Q'Q ≈ I
@test Q*L ≈ A
@test L ≈ tril(L) # it is acceptable to have small non-zero entries in L

Test Passed

## 5. Singular Value Decomposition

**Problem 5.1 (C)** Find the best rank-4 approximation (in the $2$-norm) to
$$
f(x,y) = \cos(x - y)/(x+y+1)
$$
evaluated at an evenly spaced 100 × 100 grid on the square $[0,1]^2$.

In [48]:
function bestrank4()
    # TODO: return best rank-4 approximation

    j = k = range(0,1,length = 100)
    f(x,y) = cos(x-y)/(x+y+1)

    A = [f.(j,k)]
    print(A)
    
    U, S, V = svd(A)

    U_4 = U[:, 1:4]
    V_4 = V[:,1:4]
    S_4 = Diagonal(S[1:4])

    U_4*S_4*V_4'
    
    
    
end

Fr = bestrank4()



[[1.0, 0.9801980198019803, 0.9611650485436893, 0.942857142857143, 0.925233644859813, 0.908256880733945, 0.891891891891892, 0.8761061946902655, 0.8608695652173913, 0.8461538461538461, 0.8319327731092437, 0.8181818181818181, 0.8048780487804879, 0.792, 0.7795275590551181, 0.7674418604651163, 0.7557251908396946, 0.7443609022556391, 0.7333333333333333, 0.7226277372262774, 0.7122302158273381, 0.7021276595744681, 0.6923076923076923, 0.6827586206896552, 0.673469387755102, 0.6644295302013422, 0.6556291390728477, 0.6470588235294118, 0.6387096774193548, 0.6305732484076433, 0.6226415094339623, 0.6149068322981366, 0.607361963190184, 0.6000000000000001, 0.592814371257485, 0.5857988165680473, 0.5789473684210527, 0.5722543352601156, 0.5657142857142857, 0.5593220338983051, 0.553072625698324, 0.5469613259668509, 0.540983606557377, 0.5351351351351351, 0.5294117647058824, 0.5238095238095238, 0.518324607329843, 0.5129533678756477, 0.5076923076923077, 0.5025380710659898, 0.4974874371859296, 0.49253731343283

MethodError: MethodError: no method matching zero(::Type{Vector{Float64}})
Closest candidates are:
  zero(!Matched::Union{Type{P}, P}) where P<:Dates.Period at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/periods.jl:53
  zero(!Matched::Diagonal) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/special.jl:381
  zero(!Matched::UniformScaling{T}) where T at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/uniformscaling.jl:135
  ...

In [43]:
x = 9/99
y = 10/99
@test rank(Fr) == 4
@test abs(Fr[10,11] - cos(x - y)/(x + y + 1)) ≤ 2E-6

Test Failed at /Users/noahrubin/Desktop/Imperial/Year 2/Numerical Analysis/MATH50003NumericalAnalysis/exams/practice.ipynb:4
  Expression: abs(Fr[10, 11] - cos(x - y) / (x + y + 1)) ≤ 2.0e-6
   Evaluated: 0.0013525675695882367 ≤ 2.0e-6


Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")

## 6. Differential Equations

**Problem 6.1 (A)** Complete the function `airyai(n)` that returns a length-$n$ `Vector{Float64}`
$$
\begin{bmatrix}
u_1 \\
⋮ \\
u_n
\end{bmatrix}
$$
such that $u_k$ approximates the solution to the equation
$$
\begin{align*}
u(0) &= 1 \\
u(1) &= 0 \\
u'' - x u &= 0
\end{align*}
$$
at the point $x_k = (k-1)/(n-1)$ using the second order finite-difference approximation:
$$
u''(x_k) ≈ {u_{k-1} - 2u_k + u_{k+1} \over h^2}
$$
for $k = 2, …, n-1$, in $O(n)$ operations.

In [ ]:
function airy(n)
    # TODO: return a Vector{Float64} approximating the solution to the ODE
    
end

In [17]:
u = airy(1000)
@test u[1] ≈ 1
@test u[end] ≈ 0
# this compares agianst the exact formula
@test abs(u[500] - 0.4757167332829094) ≤ 2E-8

LoadError: UndefVarError: airy not defined

In [25]:
x = 1:6



SymTridiagonal(Vector{Float64}(x[1:5]), [1.0; zeros(3)])

5×5 SymTridiagonal{Float64, Vector{Float64}}:
 1.0  1.0   ⋅    ⋅    ⋅ 
 1.0  2.0  0.0   ⋅    ⋅ 
  ⋅   0.0  3.0  0.0   ⋅ 
  ⋅    ⋅   0.0  4.0  0.0
  ⋅    ⋅    ⋅   0.0  5.0

In [21]:
x

4-element Vector{Float64}:
 1.0
 2.0
 3.0
 4.0